In [9]:
from ift6758.data.acquisition import NHLGameData

data_path_raw = './../ift6758/data/json_raw/'
nhl_games_data = NHLGameData(data_path_raw)

for year in range(2016,2021):
    nhl_games_data.fetch_season(year)

Loading from cache file ./../ift6758/data/json_raw/2016\2016-regular.pkl
Found 1230 regular games for season 2016-2017
Loading from cache file ./../ift6758/data/json_raw/2016\2016-playoff.pkl
Found 102 playoff games for season 2016-2017
Loading from cache file ./../ift6758/data/json_raw/2017\2017-regular.pkl
Found 1271 regular games for season 2017-2018
Loading from cache file ./../ift6758/data/json_raw/2017\2017-playoff.pkl
Found 105 playoff games for season 2017-2018
Loading from cache file ./../ift6758/data/json_raw/2018\2018-regular.pkl
Found 1271 regular games for season 2018-2019
Loading from cache file ./../ift6758/data/json_raw/2018\2018-playoff.pkl
Found 105 playoff games for season 2018-2019
Loading from cache file ./../ift6758/data/json_raw/2019\2019-regular.pkl
Found 1271 regular games for season 2019-2020
Loading from cache file ./../ift6758/data/json_raw/2019\2019-playoff.pkl
Found 105 playoff games for season 2019-2020
Loading from cache file ./../ift6758/data/json_raw/2

In [11]:
import ipywidgets as widgets
from ift6758.data import SeasonType

## Define notebook widgets

year_slider = widgets.IntSlider(
    min=2016, 
    max=2020, 
    description='Season :')

seasonType_selector = widgets.Dropdown(
    options=[item.name.capitalize() for item in SeasonType],
    value=SeasonType.REGULAR.name.capitalize(),
    description='Season Type :',
    disabled=False,
)

game_id_slider = widgets.IntSlider(
    min=0, 
    max=1350, 
    description='Game Index :')

game_summary_desc = widgets.HTML(
    value="Game date",
)

event_id_slider = widgets.IntSlider(
    min=0, 
    max=1350, 
    description='Event ID :')

event_desc = widgets.HTML(
    value = "Event data"
)

out = widgets.Output()

In [12]:
import json
import matplotlib.pyplot as plt
from datetime import datetime

# Initialize global variables
all_games = []
selectedGame = None


## Define useful functions for the debug tool
def get_games_for_season_and_seasonType(season_type: SeasonType, season:int):
    return nhl_games_data.data[season][season_type.name.lower()]

def update_plot(event_id = 0):
    global selectedGame
    with out:
        plt.close('all')
        plt.ioff()
        out.clear_output(wait=True)
        nhl_rink_img = plt.imread('../figures/nhl_rink.png')
        fig, ax = plt.subplots()

        ax.imshow(nhl_rink_img, extent=[-100.0,100.0,-42.5,42.5])

        if selectedGame is not None and len(selectedGame['liveData']['plays']['allPlays']) > 0:
            event = selectedGame['liveData']['plays']['allPlays'][event_id]

            dateFormat = "%Y-%m-%dT%H:%M:%SZ"
            event_date = datetime.strptime(event['about']['dateTime'], dateFormat)
            home_name = selectedGame['gameData']['teams']['home']['abbreviation']
            away_name = selectedGame['gameData']['teams']['away']['abbreviation']

            right_tag = ''
            left_tag = ''

            periodsInfo = selectedGame['liveData']['linescore']['periods']
            for period in periodsInfo:
                period_start_time = datetime.strptime(period['startTime'], dateFormat)
                period_end_time = datetime.strptime(period['endTime'], dateFormat)

                if event_date >= period_start_time and event_date <= period_end_time:
                    homeSide = period['home']['rinkSide']
                    if homeSide == 'left':
                        left_tag = home_name
                        right_tag = away_name
                    else:
                        right_tag = home_name
                        left_tag = away_name
            
            if right_tag == '':
                homeSide = periodsInfo[0]['home']['rinkSide']
                if homeSide == 'left':
                    left_tag = home_name
                    right_tag = away_name
                else:
                    right_tag = home_name
                    left_tag = away_name

            ax.set_title(f"{event['result']['description']}"
                        f"\n{event['about']['periodTime']}  -  P{event['about']['period']}"
                        f"\n{left_tag}                                              {right_tag}"
                        )
            
            coordinates = event['coordinates']
            if 'x' in coordinates:
                ax.plot(coordinates['x'], coordinates['y'], 'bo')
            display(fig)
        else:
            out.clear_output()

def update_game_summary(game_id):
    global all_games
    global selectedGame

    selectedGame = all_games[game_id]
    dateTime = selectedGame['gameData']['datetime']
    events = selectedGame['liveData']['plays']['allPlays']
    event_id_slider.value = 0
    event_id_slider.max = (len(events) - 1) if len(events) > 0 else 0
    update_event_info(0)
    game_summary_desc.value = (f'{dateTime["dateTime"]}<br>' 
        f'Game ID : {selectedGame["gamePk"]} &nbsp {selectedGame["gameData"]["teams"]["home"]["abbreviation"]} (home) vs {selectedGame["gameData"]["teams"]["away"]["abbreviation"]} (away) <br>'
        'Summary :'
        '<table>'
        '   <tr>'
        '       <th></th>'
        '       <th>Home</th>'
        '       <th>Away</th>'
        '   </tr>'
        '   <tr>'
        '       <td>Teams</td>'
        f'       <td>{selectedGame["gameData"]["teams"]["home"]["abbreviation"]}</td>'
        f'       <td>{selectedGame["gameData"]["teams"]["away"]["abbreviation"]}</td>'
        '   </tr>'
        '   <tr>'
        '       <td>Goals</th>'
        f'       <td>{selectedGame["liveData"]["linescore"]["teams"]["home"]["goals"]}</td>'
        f'       <td>{selectedGame["liveData"]["linescore"]["teams"]["away"]["goals"]}</td>'
        '   </tr>'
        '   <tr>'
        '       <td>SoG</td>'
        f'       <td>{selectedGame["liveData"]["linescore"]["teams"]["home"]["shotsOnGoal"]}</td>'
        f'       <td>{selectedGame["liveData"]["linescore"]["teams"]["away"]["shotsOnGoal"]}</td>'
        '   </tr>'
        '   <tr>'
        '       <td>SO Goals</td>'
        f'       <td>{selectedGame["liveData"]["linescore"]["shootoutInfo"]["home"]["scores"]}</td>'
        f'       <td>{selectedGame["liveData"]["linescore"]["shootoutInfo"]["away"]["scores"]}</td>'
        '   </tr>'
        '   <tr>'
        '       <td>SO Attempts</td>'
        f'       <td>{selectedGame["liveData"]["linescore"]["shootoutInfo"]["home"]["attempts"]}</td>'
        f'       <td>{selectedGame["liveData"]["linescore"]["shootoutInfo"]["away"]["attempts"]}</td>'
        '   </tr>'
        
        )

def update_event_info(event_id):
    global selectedGame
    if len(selectedGame['liveData']['plays']['allPlays']) > 0:
        event = selectedGame['liveData']['plays']['allPlays'][event_id]
        event_desc.value = '<pre id="json"> {' + json.dumps(event, indent=2, sort_keys=True) + '}</pre>'
    else:
        event_desc.value = 'No event to show'
    update_plot(event_id)



## Define observers for the widgets value change
def on_value_change(year, season_type):
    global all_games 
    global selectedGame
    all_games.clear()
    all_games = get_games_for_season_and_seasonType(SeasonType[season_type.upper()], year).copy()
    game_id_slider.value = 0
    game_id_slider.max = (len(all_games) - 1) if len(all_games) > 0 else 0
    update_game_summary(0)

def on_year_change(change):
    on_value_change(year=change['new'], season_type=seasonType_selector.value)

def on_seasonType_change(change):
    on_value_change(year=year_slider.value, season_type=change['new'])

def on_event_id_change(change):
    update_event_info(change['new'])

def on_game_id_change(change):
    update_game_summary(change['new'])


## Link observers
year_slider.observe(on_year_change, names='value')
seasonType_selector.observe(on_seasonType_change, names='value')
game_id_slider.observe(on_game_id_change, names='value')
on_value_change(year_slider.value, seasonType_selector.value)
event_id_slider.observe(on_event_id_change, names='value')


# Launch the display of the widgets
display(
    seasonType_selector, 
    year_slider, 
    game_id_slider,
    game_summary_desc,
    event_id_slider,
    out,
    event_desc)

Dropdown(description='Season Type :', options=('Regular', 'Playoff'), value='Regular')

IntSlider(value=2016, description='Season :', max=2020, min=2016)

IntSlider(value=0, description='Game Index :', max=1229)

HTML(value='2016-10-12T23:00:00Z<br>Game ID : 2016020001 &nbsp OTT (home) vs TOR (away) <br>Summary :<table>  …

IntSlider(value=0, description='Event ID :', max=357)

Output()

HTML(value='<pre id="json"> {{\n  "about": {\n    "dateTime": "2016-10-12T21:55:06Z",\n    "eventId": 1,\n    …